In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd

In [3]:
pip install mlflow

Looking in indexes: https://artifact.intuit.com/artifactory/api/pypi/pypi-intuit/simple
     |████████████████████████████████| 15.6 MB 97.4 MB/s            
     |████████████████████████████████| 77 kB 10.8 MB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 42 kB 1.3 MB/s             
     |████████████████████████████████| 170 kB 111.0 MB/s            
     |████████████████████████████████| 210 kB 131.7 MB/s            
     |████████████████████████████████| 79 kB 13.8 MB/s             
     |████████████████████████████████| 151 kB 128.5 MB/s            
     |████████████████████████████████| 63 kB 463 kB/s              
     |████████████████████████████████| 75 kB 8.9 MB/s              
  Created wheel for databricks-cli: filename=databricks_cli-0.17.3-py3-none-any.whl size=139800 sha256=45a8ac17ac5741bff8bad7fd7432f556b5e9deb10dd1df594c76cb671be8a52a
  Stored in directory: /home/ec2-user/.cache/pip/wheels/6b/8f/c8/30c419e90b4d9e

In [4]:
#from ordinaloss.utils.datasets import ImageFolder, create_transform_pipeline
from ordinaloss.utils.pretrained_models import classification_model_densenet, classification_model_resnet, classification_model_vgg
from ordinaloss.utils.utils import create_ordinal_cost_matrix


from ordinaloss.engine.model_engine import OrdinalEngine
from ordinaloss.utils.datasets import data_load

from ordinaloss.utils.loss_utils import SinimLoss, GirlsLoss, CSCELoss


import matplotlib.pyplot as plt

from torch.utils.data import DataLoader
from torch.optim import Adam, SGD
import torch
import mlflow

device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

loaded ordinaloss.utils.pretrained_models


/home/ec2-user/SageMaker/biu/ordinaloss/ordinaloss/engine/model_engine.py:13: FutureWarning: MLflow support for Python 3.6 is deprecated and will be dropped in an upcoming release. At that point, existing Python 3.6 workflows that use MLflow will continue to work without modification, but Python 3.6 users will no longer get access to the latest MLflow features and bugfixes. We recommend that you upgrade to Python 3.7 or newer.
  import mlflow


loaded ordinaloss.engine.model_engine
loaded ordinaloss.utils.datasets
loaded ordinaloss.utils.loss_utils


'cuda:0'

In [5]:
# train_transform = create_transform_pipeline(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.3) #These are hyperparameters. 
# test_transform = create_transform_pipeline() #Without color jitter for test pipeline

# train_dataset = ImageFolder("../datasets/kneeKL224/train/", transform=train_transform)
# val_dataset = ImageFolder("../datasets/kneeKL224/val/", transform = test_transform)
# test_dataset = ImageFolder("../datasets/kneeKL224/test/", transform = test_transform)
# auto_test_dataset = ImageFolder("../datasets/kneeKL224/auto_test/", transform = test_transform)

# train_loader = DataLoader(train_dataset, batch_size = 32, shuffle=True)
# test_loader  = DataLoader(test_dataset,  batch_size = 128, shuffle=False)

# loaders, _, _ = data_load("../datasets/kneeKL224/", 16)
# train_loader = loaders["train"]
# test_loader = loaders["test"]

In [6]:
from ordinaloss.utils.models_sinim import cls_model

In [7]:
#model = classification_model_vgg("vgg16", num_classes = 5) #Could also be resnet34, resnet50, resnet101, resnet152, 
#model = classification_model_vgg("vgg16", num_classes = 5) #Could also be resnet34, resnet50, resnet101, resnet152, 
# model = classification_model_densenet("densenet121", num_classes = 5) #Could also be densenet169, densenet201

model = cls_model({"net_type": "vgg", "depth":"16", "num_class":5, "pretrained":True})

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /home/ec2-user/.cache/torch/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [10]:
from ordinaloss.utils.loss_utils import GirlsLoss, SinimLoss
#model = classification_model_vgg("vgg16", num_classes = 5)

def my_lambda_scheduler(epoch):
    return (0.9 ** (epoch // 5))


# engine = OrdinalEngine(model, 
#                        loss_fn = CSCELoss(create_ordinal_cost_matrix(5)), #CSCELoss model, matan, you know the dril.
#                        device = device, #change to gpu, let me know if it doesnt work on GPU.
#                        use_lr_scheduler=True, scheduler_lambda_fn = my_lambda_scheduler,
#                        optimizer_fn=SGD, lr = 5.0e-4, weight_decay=5.0e-4)


# engine = OrdinalEngine(model, 
#                        loss_fn = SinimLoss(), #CSCELoss model, matan, you know the dril.
#                        device = device, #change to gpu, let me know if it doesnt work on GPU.
#                        use_lr_scheduler=True, scheduler_lambda_fn = my_lambda_scheduler,
#                        optimizer_fn=SGD, lr = 5.0e-4, weight_decay=5.0e-4)


# engine = OrdinalEngine(model, 
#                        loss_fn = CSCELoss(create_ordinal_cost_matrix(5)), #CSCELoss model, matan, you know the dril.
#                        device = device, #change to gpu, let me know if it doesnt work on GPU.
#                        use_lr_scheduler=True, scheduler_lambda_fn = my_lambda_scheduler,
#                        optimizer_fn=Adam, lr = 1.0e-3, weight_decay=5.0e-2)                       

In [ ]:
from itertools import product

param_grid = {
            "lr" : [1e-3, 5e-4, 5e-5],
            "batch_size" : [16,32],
            "cb_matrix_diag_value" : [1,5,10],
            "cost_distance" : [1, 2, 3],
            }

experiments_configuration = pd.DataFrame(list(product(*param_grid.values())), columns = param_grid.keys())
len(experiments_configuration)


for _, row in experiments_configuration.iterrows():
    config = row.to_dict()

    #model = classification_model_vgg("vgg16", num_classes = 5) #For newer versions of pytorch
    model = cls_model({"net_type": "vgg", "depth":"16", "num_class":5, "pretrained":True}) #For SageMaker version

    loaders, _, _ = data_load("../datasets/kneeKL224/", int(config["batch_size"]))
    train_loader = loaders["train"]
    test_loader = loaders["test"]

    cb_matrix = create_ordinal_cost_matrix(size = 5, cost_distance=config["cost_distance"], diagonal_value=config["cb_matrix_diag_value"])

    engine = OrdinalEngine(model, 
                       loss_fn = CSCELoss(cb_matrix), #CSCELoss model, matan, you know the dril.
                       device = device, #change to gpu, let me know if it doesnt work on GPU.
                       use_lr_scheduler=True, scheduler_lambda_fn = my_lambda_scheduler,
                       optimizer_fn=SGD, lr = config["lr"], weight_decay=5.0e-4)
    
    engine.train(train_loader, test_loader, n_epochs=20)

Training, epoch 1:   0%|          | 0/362 [00:00<?, ?it/s]

{'mae': 1.1443236714975846, 'loss': 4.130715316044536, 'accuracy': 0.2777777777777778}


Training, epoch 2:   0%|          | 0/362 [00:00<?, ?it/s]

{'mae': 0.7711352657004831, 'loss': 3.142421126941552, 'accuracy': 0.48429951690821255}


Training, epoch 3:   0%|          | 0/362 [00:00<?, ?it/s]

...

Training, epoch 18:   0%|          | 0/362 [00:00<?, ?it/s]

{'mae': 0.40217391304347827, 'loss': 11.11998444709225, 'accuracy': 0.6678743961352657}


Training, epoch 19:   0%|          | 0/362 [00:00<?, ?it/s]

{'mae': 0.42934782608695654, 'loss': 11.49362179622558, 'accuracy': 0.6479468599033816}


Training, epoch 19:  99%|█████████▉| 358/362 [01:28<00:00,  4.02it/s, accuracy=0.727, batch_size=5728, loss=8.92, mae=0.322]